Si se usa Colab, instalar las dependencias de tensorflow lite

In [3]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

     |████████████████████████████████| 642 kB 5.1 MB/s 
     |████████████████████████████████| 120 kB 42.4 MB/s 
     |████████████████████████████████| 840 kB 52.0 MB/s 
     |████████████████████████████████| 6.4 MB 44.3 MB/s 
     |████████████████████████████████| 10.9 MB 23.0 MB/s 
     |████████████████████████████████| 237 kB 58.5 MB/s 
     |████████████████████████████████| 3.4 MB 45.1 MB/s 
     |████████████████████████████████| 77 kB 6.3 MB/s 
     |████████████████████████████████| 87 kB 6.3 MB/s 
     |████████████████████████████████| 42.6 MB 1.2 MB/s 
     |████████████████████████████████| 1.2 MB 47.1 MB/s 
     |████████████████████████████████| 596 kB 58.0 MB/s 
     |████████████████████████████████| 1.1 MB 47.5 MB/s 
     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 99 kB 8.1 MB/s 
     |████████████████████████████████| 48.3 MB 86 kB/s 
     |████████████████████████████████| 352 kB 75.9 MB/s 
     |███████████████

Montar google drive si se usa colab.

Cambiar los directorios por los que se quieran usar para los datos de entrenamiento

In [4]:
# Montar google drive
from google.colab import drive
drive.mount('/content/gdrive')
model_output_dir = '/content/gdrive/My Drive/Colab Notebooks/ObjectDetection'
train_dir = '/content/gdrive/My Drive/Colab Notebooks/ObjectDetection/imgs_out/train'
valid_dir = '/content/gdrive/My Drive/Colab Notebooks/ObjectDetection/imgs_out/validation'
test_dir = '/content/gdrive/My Drive/Colab Notebooks/ObjectDetection/imgs_out/not_labeled'

#!ls '$train_dir'

Mounted at /content/gdrive


Importar librerías necesarias


In [5]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

Obtener los datos de entrenamiento y validación

In [6]:
train_data = object_detector.DataLoader.from_pascal_voc(
    train_dir,
    train_dir,
    ['stop', 'turn_right']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    valid_dir,
    valid_dir,
    ['stop', 'turn_right']
)

El modelo base para la detección de objetos será EfficientDet_lite0. No es el mas preciso pero es muy ligero.

In [7]:
spec = model_spec.get('efficientdet_lite0')

Entrenar el modelo con los datos obtenidos de señales de trafico.

In [13]:
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, epochs=20, validation_data=val_data)

Epoch 1/20
24/24 [==============================] - 93s 2s/step - det_loss: 1.7484 - cls_loss: 1.0930 - box_loss: 0.0131 - reg_l2_loss: 0.0630 - loss: 1.8113 - learning_rate: 0.0065 - gradient_norm: 2.1972 - val_det_loss: 1.4305 - val_cls_loss: 0.9884 - val_box_loss: 0.0088 - val_reg_l2_loss: 0.0630 - val_loss: 1.4935
Epoch 2/20
24/24 [==============================] - 55s 2s/step - det_loss: 1.3694 - cls_loss: 0.8023 - box_loss: 0.0113 - reg_l2_loss: 0.0630 - loss: 1.4324 - learning_rate: 0.0049 - gradient_norm: 3.4927 - val_det_loss: 0.7390 - val_cls_loss: 0.4589 - val_box_loss: 0.0056 - val_reg_l2_loss: 0.0630 - val_loss: 0.8021
Epoch 3/20
24/24 [==============================] - 55s 2s/step - det_loss: 0.7404 - cls_loss: 0.4357 - box_loss: 0.0061 - reg_l2_loss: 0.0630 - loss: 0.8034 - learning_rate: 0.0048 - gradient_norm: 2.3699 - val_det_loss: 0.5540 - val_cls_loss: 0.3501 - val_box_loss: 0.0041 - val_reg_l2_loss: 0.0630 - val_loss: 0.6170
Epoch 4/20
24/24 [======================

Estructura del modelo

In [12]:
print(model.summary())

Model: ""
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  multiple                  3234464   
                                                                 
 class_net/class-predict (Se  multiple                 1746      
 parableConv2D)                                                  
                                                                 
 box_net/box-predict (Separa  multiple                 2916      
 bleConv2D)                                                      
                                                                 
Total params: 3,239,126
Trainable params: 3,191,990
Non-trainable params: 47,136
_________________________________________________________________
None


Evaluación del modelo antes de ser exportado

In [ ]:
model.evaluate(val_data)

1/1 [==============================] - 7s 7s/step



{'AP': 0.7273665,
 'AP50': 1.0,
 'AP75': 1.0,
 'AP_/stop': 0.65683407,
 'AP_/turn_right': 0.79789895,
 'APl': -1.0,
 'APm': 0.7277677,
 'APs': -1.0,
 'ARl': -1.0,
 'ARm': 0.78333336,
 'ARmax1': 0.7361111,
 'ARmax10': 0.78333336,
 'ARmax100': 0.78333336,
 'ARs': -1.0}

Esportar el modelo

In [ ]:
model.export(export_dir=model_output_dir, tflite_filename='trafficDetection.tflite')

Evaluación de resultados del modelo despues de ser exportado

In [ ]:
model.evaluate_tflite('trafficDetection.tflite', val_data)

18/18 [==============================] - 53s 3s/step



{'AP': 0.7218716,
 'AP50': 1.0,
 'AP75': 0.95352036,
 'AP_/stop': 0.63359815,
 'AP_/turn_right': 0.81014514,
 'APl': -1.0,
 'APm': 0.7218716,
 'APs': -1.0,
 'ARl': -1.0,
 'ARm': 0.7416667,
 'ARmax1': 0.7416667,
 'ARmax10': 0.7416667,
 'ARmax100': 0.7416667,
 'ARs': -1.0}

Si se está usando Colab, para descargar el modelo directamente al PC

In [ ]:

from google.colab import files
files.download('trafficDetection.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>